## YouTube Transcript API

- 影片原本就得有字幕才行
- 翻譯預設是英文，可以查看有哪些語言可以翻譯
- 影片過長的話，建議將index persist to disk

In [ ]:
# https://github.com/jdepoix/youtube-transcript-api

In [ ]:
%pip install -q llama-hub-youtube-transcript
%pip install -q llama-index-readers-youtube-transcript

In [2]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

youtube_links = [
    # "https://www.youtube.com/watch?v=zIwLWfaAg-8",  # TED
    "https://www.youtube.com/watch?v=fkHA6CUF5VQ"  # 錢線百分百
]

In [3]:
ytb_id = youtube_links[0].split("?v=")[1]
print(ytb_id)

fkHA6CUF5VQ


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
transcript_list = YouTubeTranscriptApi.list_transcripts(ytb_id)
language_code = ""
for transcript in transcript_list:
    language_code = transcript.language_code
print(language_code)

zh-TW


In [5]:
transcript_list.find_transcript([language_code])

In [ ]:
loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=youtube_links,
    languages=[language_code]
)
print(documents)

In [7]:
print(documents[0].text.replace("\n", ", "))

歡迎回到錢線百分百, 講到台北股市這個禮拜, 漲了快要接近千點的行情, 最後雖然說在禮拜四禮拜五, K棒收黑K, 那今天開高走低 最後跌了有74點, 收在23193點, 但是今天看到了外資繼續買超台股, 雖然買超縮小 只剩下20億, 但這個禮拜連五買, 至少12月行情的一開始 外資回來了, 不過外資買超 投信卻賣了, 逢高看到投信在轉賣, 投信到底現在是要繼續作帳, 還是準備結帳, 最近投信在賣, 到底賣怎麼回事的呢, 待會一一解析, 我們先看到今天的外資買超, 外資買超的前10名個股, 先來看看這些個股今天, 第一名又是復華台灣科技優息, 這檔個股前幾天, 創下了百萬張的巨量, 因為他宣布的配息, 越來越少, 這樣的配息情況, 讓散戶傷心 不要了, 結果外資一直買, 連買了好多天 每天都是第一名, 今天又是33000多張, 群創 新光金 元大滬深300正2, 華航 富邦金 緯創 凱基金, 還有美債20年正2 外資有在買, 好 講到外資買超, 先請教博傑了, 怎麼看今天外資的買超動作, 特別是復華台灣科技優息, 我們發現到了, 我們也做一個資料告訴大家, 這五天 連續五天的進出動作當中, 外資在大家賣掉高股息ETF的時候, 在大家失望利息那麼少的時候, 賣掉的時候, 外資一直給你偷偷買, 最近買好多張你看, 外資這五天買最多就是, 77萬張的00929復華台灣科技優息, 00940元大台灣價值高息, 00934中信成長高股息, 都是買超前5名, 那剛剛個股當中也有華航 凱基金, 好 到底這幾檔外資買超股, 我們要怎麼看, 這兩檔很強, 是不是可以追, 那這些ETF 高股息的ETF, 投資朋友現在賣, 會不會賣在阿呆股 請教博傑, 好 第一個外資會買這個, 復華台灣科技優息, 我覺得很標準你丟我撿, 就是如果把那個ETF的K線打出來, 你會發現, 經過了一段跌勢以後, 因為高股息的股票, 最近表現都不好, 這一段時間市場的主流, 除了AI就是中小型個股對不對, 所以高股息這一些表現不好, 那相對的他在波段的位置來講, 就是在相對低檔, 那剛好又出了這個, 配息只配0.05元 這個單月的狀況, 在這裡我覺得外資是進來你丟我撿, 那另外還有一個就是說, 最近大盤上來對不對, 那比較保守的外資他不知道買什麼, 我乾脆就買高股息, 買低的, 對 我就買低的 我就乾脆, 所以你

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

In [9]:
nodes = list(index.docstore.docs.values())
print("Node count: ", len(nodes))

Node count:  29


In [10]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:3b", request_timeout=600)

In [11]:
query_engine = index.as_query_engine(llm=llm)
res = query_engine.query("請用繁體中文總結這篇文章的內容")
print(res)

該文章主要討論了台股投資和短線交易，提到了幾個 stocks 的行情分析。作者首先介紹了兆利和新日興的簡要信息，後來進一步分析了金山電的行情，強調了這家公司的 BBU 超級電容概念以及其在一次漲速過程中的重要性。

作者認為，這次漲速的機會主要是從基本面的角度來看，而不是因為業績出現明顯改善。同時也提到了消費性電子的部分有轉機題材，尤其是在位階低的產品上。

其後，作者進一步分析了幾個 stocks 的短線交易機會，包括兆利、金山電和華星光。對於兆利和金山電，作者認為短線買點是這邊，特別注意到兆利在過去的低點出現了一個新的漲速機會。對於華星光，作者認為該公司今年來行情相對較弱，但仍有機會。

最後，作者提到了對於台股投資的建議，特別是建議選擇12月剛剛要轉強的stocks，而不是漲得高高的stocks。同時也強調了流量和人氣對於短線交易的重要性。
